In [ ]:
# Imports
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from tqdm.notebook import tqdm
from DQNagents import DQNagent, DOE
from System import System

# Test-System

## Systemaufbau

Das Test-System besteht aus einer nichtlinearen Regelstrecke (PT1-Glied). Deren Ausgang wird durch eine Störung (ebenfalls verzögert durch ein PT1-Glied) beeinflusst. Geregelt werden soll die Ausgangsgröße $y$ des Gesamtsystems. Die relevanten Übertragungsfunktionen lauten:

$$ F_S = \frac{y_S}{u_S}= \frac{K(y_S)}{1 + T_Sp} $$
$$ F_Z = \frac{y_Z}{u_Z} = \frac{1}{1 + 10T_Sp} $$
mit
\begin{align}
    K(y_S) &= 0.1\cdot y_S + 2 \\
    -5 &\leq u_S \leq 5 \\
    -2 &\leq u_Z \leq 2 \\
    y &= y_S + y_Z \\
    -8.5 &\leq y \leq 21.5
\end{align}

Folgende größen werden als messbar angenommen:
- $y$
- $y_S$
- $u_S$
- $u_Z$

<img src="../SFB.png" alt="Signalflussbild des Gesamtsystems" title="Signalflussbild des Gesamtsystems" />

### Herleitung der Solver-Gleichung
Z-Transformation der Übertragungsfunktion für PT1-Glieder:
\begin{align}
    F(p) = \frac{y}{u}= \frac{K}{1 + T\cdot p} \\
    F(z) = \frac{y_k}{u_k} = K \cdot \frac{1-a}{z-a}
\end{align}
mit
$$a = e^{-\frac{dt}{T}}$$

Auflösen nach $y_k$:
\begin{align}
    F(z) = \frac{y_k}{u_k} &= \frac{K-Ka}{z-a} \\
    y_k \cdot (z - a) &= u_k \cdot (K - Ka) \\
    y_{k+1} - ay_k  &= u_k \cdot (K - Ka) \\
    y_{k+1} - ay_k  &= u_k \cdot (K - Ka) \\
    y_{k+1} &= u_k \cdot (K - Ka) + ay_k
\end{align}

## Implementierung des System-Modells

In [ ]:
sysMax = System(0, probZ=0)
sysMin = System(0, probZ=0)
for i in range(10000):
    sysMin.uZ = -2
    sysMax.uZ = 2
    yMin = sysMin.step(-5)
    yMax = sysMax.step(5)
    if i % 2500 == 0:
        print("yMin: {0:.2f}; yMax: {1:.2f}".format(yMin, yMax))

wMin = np.round(0.7*yMin, 1)
wMax = np.round(0.5*yMax, 1)

### Test-System und Ergebnisdarstellung

In [ ]:
# Prepare test system
plant = System(0.0)

# Regler
Der selbstlernende Regler wird mittels Deep-Q-Learning Realisiert. Die Implementierung erfolgt in Anlehnung an das DQN Tutorial von pytorch sowie an das entsprechende DeepMind Paper (Lernen von Atari Spielen). Das DQN verfahren ist anders als NFQ lernen ein online Verfahren. Es scheint die Weiterentwicklung des NFQ Verfahrens zu sein und zielt auf eine bessere konvergenz der zu trainierenden Modelle ab. Diese ist insbesondere bei großen KNN von Bedeutung. Daher wurde von DeepMind das Bsp. Atari-Spiele gewählt. Wobei der Input die Pixel-Daten der Spielegrafik waren und entsprechende Faltungsschichten selbstständig die wesentlichen merkmale bestimmen mussten. Da zeitlich korellierte Daten schlecht für das Training sind, werden eine Vielzahl an Datensätzen in einem Speicher (Replay-Memory) abgelegt. Zu jedem Prozessschritt werden eine kleine Anzahl an zufälligen Datenpunkten aus dem Speicher gezogen und das Modell trainiert. Dabei wird das Modell doppelt verwendet. Einmal als in jedem Schritt trainiertes und für die Handlungen konsultierte Modell. Das andere soll Q-werte, welche für das Training als Referenzwerte dienen, vorhersagen. Letzteres wird nur zu wenigen Schritten verbessert. Dabei werden die aktuellen Parameter (Gewichte, etc.) des anderen Modells kopiert.

Ein System-Zustand ist durch die Messbaren signale gekennzeichnet: $w, y, y_S, u_S, u_Z$

## Agent mit Erfahrungsspeicher und -modell

In [ ]:
newAgent = True
# hyper parameter
nEpoch = 5000
nBatch = 100
batchSize = 32
probW = 0.5 # probability for change of set point

memCapacity = 320000

eps0 = 1.
epsDecay = 500
epsMin = 0.05

targetUpdate = 250  # rate of epochs for updating target model

# max cost per action
cMax = 0.1
mu = 2.5  # target range for cost function

# init agent
if newAgent:
    agent = DQNagent(6, 7, memCapacity, batchSize,
                    eps0, epsMin, epsDecay, targetUpdate)
actuator = DOE(plant.uSmin, plant.uSmax)

visualisation = True

# define helper functions
A_CONV = [plant.uSmin * 0.5, plant.uSmin * 0.1, plant.uSmin * 0.05,
          0.,
          plant.uSmax * 0.05, plant.uSmax * 0.1, plant.uSmax * 0.5]

def getState(w, uS, y, plant, actuator):
    return np.array([w, y, plant.yS, uS, plant.uZ, actuator.state])


def fillRM(system, actuator, agent, memCapacity, nActions, QbMin):
    for _ in range(memCapacity):
        aIdx = np.random.randint(nActions)
        u = actuator.step(A_CONV[aIdx])
        y = system.getEq(u)
        state = getState(y, u, y, system, actuator)
        agent.remember(state, aIdx, QbMin, state, 0.)

In [ ]:
# different cost functions
def costSimple(w, y, eps=0.5):
    if np.abs(w-y) < eps:
        return 0
    else:
        return 0.01


def costSmooth(w, y, cMax=0.01, mu=0.5):
    e = np.abs(w-y)
    omega = np.tanh(np.sqrt(0.95) / mu)

    return np.tanh(e*omega)**2. * cMax

getCosts = lambda w, y: costSmooth(w, y, cMax=cMax, mu=mu)

def getReward(w, y, w_last, y_last, cMax=0.01):

    if (abs(w - y) > 2.5) & (abs(w - y) < abs(w_last - y_last)):
        reward = cMax / 2.
    else:
        reward = 0
    return reward

## Training

In [ ]:
if visualisation:
    # prepare result visualisation
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.02)
    fig.update_xaxes(title_text="time in s", row=2, col=1)
    fig.update_yaxes(row=1, col=1, title_text="plant values", range=[-9, 22.5])
    fig.update_yaxes(row=2, col=1, title_text="costs", range=[0., cMax])

    # data
    time = np.arange(nBatch) * plant.dt
    w = np.zeros(nBatch)
    u = np.zeros(nBatch)
    y = np.zeros(nBatch)
    c = np.zeros(nBatch)

    # line objects
    lineU = go.Scatter({"x": time,
                        "y": u,
                        "name": "u",
                        "uid": "uid_lineU",
                        "yaxis": "y1",
                        "line": {"color": "#54A24B",
                                "width": 1
                                }
                        })
    lineW = go.Scatter({"x": time,
                        "y": w,
                        "name": "w",
                        "uid": "uid_lineW",
                        "yaxis": "y1",
                        "line": {"color": "#4C78A8",
                                "dash": "dash",
                                "width": 1
                                }
                        })
    lineY = go.Scatter({"x": time,
                        "y": y,
                        "name": "y",
                        "uid": "uid_lineY",
                        "yaxis": "y1",
                        "line": {"color": "#4C78A8",
                                "width": 1
                                }
                        })
    lineC = go.Scatter({"x": time,
                        "y": c,
                        "name": "c",
                        "uid": "uid_lineC",
                        "yaxis": "y1",
                        "line": {"color": "#000000",
                                "width": 1
                                }
                        })

    fig.update_layout(title='', height=800)
    # add lines to plots
    fig.add_trace(lineU, row=1, col=1)
    fig.add_trace(lineW, row=1, col=1)
    fig.add_trace(lineY, row=1, col=1)
    fig.add_trace(lineC, row=2, col=1)
    # create widget
    fWidget = go.FigureWidget(fig)
    # get direct connection to line data (overwrite variables used for lines)
    lineU = fWidget.data[0]
    lineW = fWidget.data[1]
    lineY = fWidget.data[2]
    lineC = fWidget.data[3]

In [ ]:
# RUN TRAINING
if visualisation:
    display(fWidget)
# save last reward and training loss of each epoch
cEnd = np.zeros(nEpoch)
loss = np.zeros(nEpoch)
# observe agents min and max Q-Values
Qmin = np.zeros(nEpoch)
Qmax = np.zeros(nEpoch)

wStep = np.round(wMin + (wMax-wMin)*np.random.random(), 1)

# fillRM(plant, agent, memCapacity, nActions, QbMin)
for epoch in tqdm(range(nEpoch)):
    # Initialize the environment and state
    plant.reset()
    actuator.reset()
    uStep = 0
    if np.random.random() < probW:
        wStep = np.round(wMin + (wMax-wMin)*np.random.random(), 1)
    yStep = np.round(yMin + (yMax-yMin)*np.random.random(), 1)
    plant.yS = yStep
    costs = 0
    # get init state
    NNstate = getState(wStep, uStep, yStep, plant, actuator)

    cEpoch = 0
    for batch in range(nBatch):
        # Select and perform an action
        # use epoch for exploration -> later epochs should be dominated by DQN
        aIdx = agent.act(NNstate)
        uStep = actuator.step(A_CONV[aIdx])
        yStep = plant.step(uStep)

        # get immediate costs
        costs = getCosts(wStep, yStep)
        reward = getReward(wStep, yStep, NNstate[0], NNstate[1])
        costs -= reward
        if costs < 0.:
            costs = 0.
        
        cEpoch += costs

        # get new step in state
        NNstateNext = getState(wStep, uStep, yStep, plant, actuator)

        if batch + 1 < nBatch:
            # Store the transition in memory
            agent.remember(NNstate, aIdx, costs, NNstateNext, 0.)
            # Train model ANN
            agent.replay(epoch)
            # save state for next iteration
            NNstate = np.copy(NNstateNext)

        if visualisation:
            # update vis data
            u[batch] = uStep
            w[batch] = wStep
            y[batch] = yStep
            c[batch] = costs

    agent.remember(NNstate, aIdx, costs, NNstateNext, 1.)

    # train agent model
    loss[epoch], Qb = agent.replay(epoch, True, epoch==1000)
    Qmin[epoch] = Qb[0]
    Qmax[epoch] = Qb[1]

    cEnd[epoch] = cEpoch

    # update visualisation
    if visualisation:
        with fWidget.batch_update():
            lineU.x = time
            lineW.x = time
            lineY.x = time
            lineC.x = time
            lineU.y = u
            lineW.y = w
            lineY.y = y
            lineC.y = c
            fWidget.layout['title'] = "Epoch: {}".format(epoch)
        # reset vis data
        y *= 0
        w *= 0
        u *= 0
        c *= 0
    else:
        if (epoch > 0) & (epoch % 500 == 0):
            print("Current Costs trend: {:.2f}, "
                  "Std. Deviation in Costs: {:.2f}, "
                  "Qmin: {:.2f}, Qmax: {:.2f}"
                  .format(cEnd[epoch-500:epoch].mean(), 
                          cEnd[epoch-500:epoch].std(), 
                          Qb[0], Qb[1]))


In [ ]:
# show cost curve and training loss over all epochs
# get mean costs to see trends
w = 50
cEndM = np.convolve(cEnd, np.ones(w), 'full') / w

fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)
fig.update_xaxes(title_text="number of Epoch", row=2, col=1)
fig.update_yaxes(row=1, col=1, title_text="Costs", autorange=True)
fig.update_yaxes(row=2, col=1, title_text="ANN Loss", autorange=True)

xEpochs = np.arange(nEpoch)
lineCostsEnd = go.Scatter({"x": xEpochs,
                           "y": cEnd,
                           "name": "costs",
                           "opacity": 0.25,
                           "uid": "uid_rEndLine",
                           "yaxis": "y1",
                           "line": {"color": "#000000",
                                    "width": 1
                                    }
                           })
lineCostsEndM = go.Scatter({"x": xEpochs,
                           "y": cEndM,
                           "name": "costs mean",
                           "uid": "uid_rEndLine",
                           "yaxis": "y1",
                           "line": {"color": "#000000",
                                    "width": 1
                                    }
                           })
lineQmin = go.Scatter({"x": xEpochs,
                       "y": Qmin,
                       "name": "Qmin",
                       "uid": "uid_lineQmin",
                       "yaxis": "y1",
                       "line": {"color": "#CC4F4F",
                                "width": 1
                                }
                       })
lineQmax = go.Scatter({"x": xEpochs,
                       "y": Qmax,
                       "name": "Qmax",
                       "uid": "uid_lineQmax",
                       "yaxis": "y1",
                       "line": {"color": "#1E70CC",
                                "width": 1
                                }
                       })
lineLoss = go.Scatter({"x": xEpochs,
                       "y": loss,
                       "name": "loss",
                       "uid": "uid_lineLoss",
                       "yaxis": "y1",
                       "line": {"color": "#D83C20",
                                "width": 1
                               }
                       })
fig.update_layout(title='', height=800)
fig.add_trace(lineCostsEnd, row=1, col=1)
fig.add_trace(lineCostsEndM, row=1, col=1)
fig.add_trace(lineQmin, row=1, col=1)
fig.add_trace(lineQmax, row=1, col=1)
fig.add_trace(lineLoss, row=2, col=1)
cEndWidget = go.FigureWidget(fig)
cEndWidget

In [ ]:
# test model
nTest = 6000  # number of test steps
# switch of exploration
agent.EpsilonEnd = 0
agent.EpsilonStart = agent.EpsilonEnd
agent.Epsilon = 0

# data for visualisation
time = np.arange(nTest) * plant.dt / 60.
w = np.zeros(nTest)
u = np.zeros(nTest)
y = np.zeros(nTest)

# prepare some set points
wTest = [0.25*wMax, 0.5*wMax, 0.5*wMin, 0.75*wMin, 
         0.1*wMax, 0.25*wMin, 0.75*wMax, 0]
wStep = int(np.round(nTest / len(wTest)))
for i, wt in enumerate(wTest[:-1]):
    w[i*wStep:(i+1)*wStep] = wt
w[(i+1)*wStep:] = wTest[-1]
# reset system
plant.reset()
actuator.reset()
# get init state
uStep = 0
wStep = w[0]
yStep = plant.yS + plant.yZ
NNstate = getState(wStep, uStep, yStep, plant, actuator)
for stepIdx in range(nTest):
    wStep = w[stepIdx]
    aIdx = agent.act(NNstate)
    uStep = actuator.step(A_CONV[aIdx])
    yStep = plant.step(uStep)
    # update state
    NNstate = getState(wStep, uStep, yStep, plant, actuator)

    # update vis data
    y[stepIdx] = yStep
    u[stepIdx] = uStep

fig = go.Figure()
fig.add_trace(go.Scatter({"x": time,
                          "y": u,
                          "name": "u",
                          "uid": "uid_lineU",
                          "line": {"color": "#54A24B",
                                   "width": 1
                                  }
                          }))
fig.add_trace(go.Scatter({"x": time,
                          "y": w,
                          "name": "w",
                          "uid": "uid_lineW",
                          "line": {"color": "#4C78A8",
                                   "dash": "dash",
                                   "width": 1
                                  }
                          }))
fig.add_trace(go.Scatter({"x": time,
                          "y": y,
                          "name": "y",
                          "uid": "uid_lineY",
                          "line": {"color": "#4C78A8",
                                   "width": 1
                                  }
                          }))
fig.update_xaxes(title_text="Time in Min")
fig.update_yaxes(title_text="Control Values", autorange=True)

In [ ]:
agent.saveModel()